# Calculation general flare distributions in $\mu$

In [1]:
import numpy as np
import scipy.special as sc
import 


In [2]:
# general distribution for area 
def rice_dist(x, sigma, nu):
    x = np.abs(x)
    return x / sigma**2 * np.exp( - (x**2 + nu **2 ) / (2 * sigma **2)) * sc.iv(0, (x * nu / sigma**2))

sigma_fit, nu_fit = (9.02769086, 10.48768514)

def rho(alpha):
    return rice_dist(alpha, sigma_fit, nu_fit) / 2 / np.pi / np.cos(alpha) / 2
            # fit of dist when integrated over phi and absolute value so accounting for that

Therefore density is $$\rho_\mu(\mu) = \frac{2}{\sin \theta} \int_{[0, 2\pi]}d\phi \rho (\alpha(\theta, \phi))
.$$
where
$$
\cos \alpha = \sin \tau \sin \theta \cos \phi +\cos \theta \cos \tau  $$

In [ ]:
# general distribution for given tau

def get_alpha(tau, theta, phi):
    return np.arccos( np.sin( tau ) * np.sin(theta) * np.cos(phi) + np.cos(tau) * np.cos(theta))

def rho_mu(mu):
     
    
